In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

In [ ]:
from astartes import train_val_test_split

In [ ]:
params = Chem.SmilesParserParams()
params.removeHs = False

# Read in the data
- This csv file comes from `data.tar.gz` from [Chemprop](https://github.com/chemprop/chemprop). It stores the QM9 data which was published in the following publication: Ramakrishnan, R., Dral, P.O., Rupp, M. and Von Lilienfeld, O.A. "Quantum Chemistry Structures and Properties of 134 kilo Molecules. In: Sci. Data 1.1 (2014), pp.1-7. [link](https://www.nature.com/articles/sdata201422) 

In [ ]:
CSV_PATH = 'qm9.csv'
df = pd.read_csv(CSV_PATH)

In [ ]:
df.describe()

# Random splits

In [ ]:
RANDOM_SPLITS = []
sampler = "random"
for seed in range(5):
    # create 85:5:10 data split
    _, _, _, train_indices, val_indices, test_indices = train_val_test_split(
        np.arange(len(df)),
        train_size=0.85,
        val_size=0.05,
        test_size=0.1,
        sampler=sampler,
        random_state=seed,
        return_indices=True,
    )
    RANDOM_SPLITS.append([train_indices, val_indices, test_indices])

In [ ]:
with open('QM9_splits/QM9_splits_random.pkl', 'wb') as f:
    pkl.dump(RANDOM_SPLITS, f)

# Scaffold splits

In [ ]:
SCAFFOLD_SPLITS = []
sampler = "scaffold"
for seed in range(5):
    # create 85:5:10 data split
    (
        _,
        _,
        _,
        train_labels,
        val_labels,
        test_labels,
        train_indices,
        val_indices,
        test_indices,
    ) = train_val_test_split(
        df.smiles.values,
        train_size=0.85,
        val_size=0.05,
        test_size=0.1,
        sampler=sampler,
        random_state=seed,
        return_indices=True,
    )
    SCAFFOLD_SPLITS.append([train_indices, val_indices, test_indices])

In [ ]:
with open('QM9_splits/QM9_splits_scaffold.pkl', 'wb') as f:
    pkl.dump(SCAFFOLD_SPLITS, f)

# KMeans

Featurize the data using morgan fingerprint with standard settings

Function taken from Chemprop: https://github.com/chemprop/chemprop/blob/master/chemprop/features/features_generators.py

In [ ]:
MORGAN_RADIUS = 2
MORGAN_NUM_BITS = 2048

def morgan_counts_features_generator(
    mol,
    radius=MORGAN_RADIUS,
    num_bits=MORGAN_NUM_BITS,
):
    """
    Generates a counts-based Morgan fingerprint for a molecule.
    :param mol: A molecule (i.e., either a SMILES or an RDKit molecule).
    :param radius: Morgan fingerprint radius.
    :param num_bits: Number of bits in Morgan fingerprint.
    :return: A 1D numpy array containing the counts-based Morgan fingerprint.
    """
    mol = Chem.MolFromSmiles(mol) if type(mol) == str else mol
    features_vec = AllChem.GetHashedMorganFingerprint(mol, radius, nBits=num_bits)
    features = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(features_vec, features)

    return features

In [ ]:
morgan_fps = np.zeros((len(df), MORGAN_NUM_BITS))
for i, row in df.iterrows():
    rmol = Chem.MolFromSmiles(row.smiles, params)
    morgan = morgan_counts_features_generator(rmol)
    morgan_fps[i, :] = morgan

In [ ]:
KMEANS_SPLITS = []
sampler = "kmeans"
for seed in range(5):
    # create 85:5:10 data split
    _, _, _, _, _, _, train_indices, val_indices, test_indices = train_val_test_split(
        morgan_fps,
        train_size=0.85,
        val_size=0.05,
        test_size=0.1,
        sampler=sampler,
        hopts={"n_clusters": 100},
        random_state=seed,
        return_indices=True,
    )
    KMEANS_SPLITS.append([train_indices, val_indices, test_indices])

In [ ]:
with open('QM9_splits/QM9_splits_kmeans.pkl', 'wb') as f:
    pkl.dump(KMEANS_SPLITS, f)